In [77]:
import pandas as pd
import pdb
import requests
import json

static_path=""

In [78]:
df = pd.read_csv("test.csv")

In [79]:
df

,Unnamed: 0,id,group,querystr,ai_train_label,ai_predict_label(pf),ai_predict_score(pf),ai_predict_label,ai_predict_score,ai_predict_candidate
0,0,c3b94e,0,JPWO2020040314,BATERY/CYL,BATERY/CYL,2.578713,BATERY/CYL,0.998880,"{""BATERY/CYL"": ""0.998880"", ""COM-M/ANTENA"": ""0...."
1,1,e480b8,0,WO2022168622,BATERY/CYL,BATERY/CYL,3.117879,BATERY/CYL,0.998393,"{""BATERY/CYL"": ""0.998393"", ""COM-M/ANTENA"": ""0...."
2,2,9e93e1,0,JPWO2021024942,BATERY/CYL,BATERY/CYL,3.341799,BATERY/CYL,0.999108,"{""BATERY/CYL"": ""0.999108"", ""COM-M/ANTENA"": ""0...."
3,3,8e6a2b,0,WO2022163480,BATERY/CYL,BATERY/CYL,3.062766,BATERY/CYL,0.998281,"{""BATERY/CYL"": ""0.998281"", ""COM-M/ANTENA"": ""0...."
4,4,8a6118,0,WO2022163049,BATERY/CYL,BATERY/CYL,3.040922,BATERY/CYL,0.996804,"{""BATERY/CYL"": ""0.996804"", ""COM-M/CONECT"": ""0...."
...,...,...,...,...,...,...,...,...,...,...
233,233,003fd1,4,JP2012191318,COM-M/ANTENA,COM-M/ANTENA,1.787056,COM-M/ANTENA,0.784006,"{""COM-M/ANTENA"": ""0.784006"", ""COM-M/CONECT"": ""..."
234,234,a346c4,4,JP2009182793,COM-M/ANTENA,COM-M/ANTENA,2.054523,COM-M/ANTENA,0.986647,"{""COM-M/ANTENA"": ""0.986647"", ""BATERY/CYL"": ""0...."
235,235,5be9d1,4,JP2015144341,COM-M/ANTENA,COM-M/ANTENA,1.518284,COM-M/ANTENA,0.862570,"{""COM-M/ANTENA"": ""0.862570"", ""COM-M/CONECT"": ""..."
236,236,e7695a,4,JPWO2018216453,COM-M/ANTENA,COM-M/ANTENA,1.166323,COM-M/CONECT,0.647576,"{""COM-M/CONECT"": ""0.647576"", ""COM-M/ANTENA"": ""..."


In [80]:
import pandas as pd
import tqdm as tqdm
import numpy as np
import pdb
from typing import Tuple
from sklearn.preprocessing import LabelEncoder

from time import time
from typing import Tuple
from tqdm import tqdm

from janome.tokenizer import Tokenizer
from collections import defaultdict
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from janome.tokenfilter import POSKeepFilter


def tfidf(df,head):
    
    t0 = time()

    patentid_list=[] #特許のID配列
    tfidf_word=[] #TFIDFの語句配列
    tfidf_score=[] #TFIDFのスコア配列
    train_label=[]

    for id,tfidf in zip(df.patent_id,df.tfidf):
        df1 = pd.DataFrame(list(tfidf.items()),columns= ['word','tfidf'])
        df1['tfidf']= df1['tfidf'].astype(float)
       
        #df1 = df1[df1['tfidf']> head ]
        df1 = df1.sort_values(by='tfidf',ascending=False).head(head)
        patentid_list.append(id)
        tfidf_word.append(df1['word'].tolist())
        tfidf_score.append(df1['tfidf'].tolist())
    
    train_label = df['ai_train_label'].tolist()

    unique_word = set([x for row in tfidf_word for x in row])
    unique_idx = {w:i for i,w in enumerate(unique_word)}

    print('number of unique words = {}'.format(len(unique_word)))
    print('number of data lengths = {}'.format(len(tfidf_word)))

    to_unique_idx = np.vectorize(unique_idx.get)
    tfidf_arr = np.zeros((len(tfidf_word),len(unique_word)),dtype='float32')

    for i,(words,scores) in enumerate(zip(tfidf_word,tfidf_score)):
        words = np.array(words)
        scores = np.array(scores,dtype=float)
    
        try:
            idx = to_unique_idx(words)
            tfidf_arr[i,idx] = scores
        except:
            pass

# 変換前のデータ
    train_arr = np.array(train_label)
    le = LabelEncoder()
    le.fit(train_arr)
    label_names = le.classes_
    label_array = le.transform(train_arr)
    
    waist_time = time() - t0
    print("tfidf time: %0.3fs" % waist_time)

    unique_words = np.array(list(unique_word))

    return tfidf_arr, train_arr, label_names , unique_words,label_array


def get_token(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    
    word = ""
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞":
            word +=token.surface + " "
        if part_of_speech == "動詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容動詞":
            word +=token.base_form+ " "
    return word


def tfidf_calc(target,token,max_word):
    corpus=[]
    
    for item in tqdm(target): 
        token=get_token(item)
        corpus.append(token)

    vectorize = TfidfVectorizer(use_idf=True,max_df=0.9)
    
    tfidf = vectorize.fit_transform(corpus)
    
    output =[]
    words = vectorize.get_feature_names()
    for doc_id,vec in zip(range(len(corpus)),tfidf.toarray()):
        jstring=""
        for w_id,tfidf in sorted(enumerate(vec),key=lambda x:x[1],reverse=True)[:max_word]:
            lemma= words[w_id]
            jstring += '"{}"=>"{:.7g}",'.format(lemma,tfidf*100)
        output.append(jstring)
    
    return output


In [81]:
def post_patent_list(body):
    if body:
        data = body['patent']
        tfidf_max_length= body['tfidf_max_length']
        numofword = body['numofword']
        df = pd.read_json(data)##CrossValidationで使うためにデータフレームにする
        patent_list = df['querystr'].tolist()##クエリストリングをリストにする
       
        #item_patent = cruds.search_patents(
        #    db=db, mode=mode, patent_numbers=patent_list)
                
        # modelスケマからjson→pandasに変換する
        json_item_data = jsonable_encoder(item_patent)
        df_s = pd.DataFrame(json_item_data)
        
        for index,row in df_s.iterrows():
            label = df[df.querystr == row.koukai_number].ai_train_label
            df_s.at[index,'ai_train_label'] = label.values[0]
            
        # pickleにつける
        df_s.to_pickle(static_path + body['token'])

       # リダイレクト
        return "/train_tfidf/?token={}&tfidf_max_length={}&numofword={}".format(body['token'], tfidf_max_length,numofword)

staric_path= ""
    
def post_train_tfidf(unpickled_df, tfidf_max_length,token):
    
    tfidf_arr, label_arr,label_names ,unique_word,label_array= tfidf(unpickled_df, tfidf_max_length)
    # pickleにつける

    np.save(static_path +'tfidf_arr.npy', tfidf_arr)
    np.save(static_path +'label_arr.npy', label_arr)
    np.save(static_path +'label_array.npy', label_array)
    np.save(static_path +'label_names.npy', label_names)
    np.save(static_path + 'unique_names.npy', unique_word)
    # リダイレクト
    return {"token": token, "tfidf_arr": tfidf_arr.shape, "label_arr": label_arr.shape, "label_names": label_names.shape, "unique_word": unique_word.shape}


In [82]:
#絞り込み後の特許リストで交差検証を実施するルーチン
def mlFlow(df,token,tfidf_target,num):
    url ="http://172.26.106.79:7999/patent_list2"
#JSON形式のデータ
    df.to_pickle(static_path + 'demo')
    json_data = {
    'patent': df.to_json(),
    'token':token,
    'tfidf_max_length':tfidf_target
    }    
    
    with open('patent_number.json', 'w') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

#POST送信
    try:
        res = requests.post( url,  json = json_data)   #dataを指定する    
        json_item_data = res.json()
    except:
        return "error"

    return json_item_data

In [83]:
json_data = mlFlow(df,"demo",100,5) #patent number
df2 = pd.DataFrame(json_data)

In [84]:
#df2

In [85]:
for index,row in df2.iterrows():
    label = df[df.querystr == row.koukai_number].ai_train_label
    df2.at[index,'ai_train_label'] = label.values[0]
df2.to_pickle("demo")
df2.head()
df2.to_csv("tfidf.csxv")

In [87]:
result = post_train_tfidf(df2,200,"demo") #tfidf dataframe

number of unique words = 9751
number of data lengths = 238
tfidf time: 0.376s


In [88]:
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,LeaveOneOut, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import Perceptron
from time import time
from tqdm import tqdm
from urllib3 import HTTPResponse
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from sklearn.ensemble import RandomForestClassifier
from fastapi import APIRouter, Depends, File, HTTPException, Query, UploadFile
from fastapi.encoders import jsonable_encoder
from fastapi.responses import StreamingResponse
from flask import jsonify
from sklearn.feature_extraction import img_to_graph
from sqlalchemy.orm import Session
import requests
from fastapi.responses import RedirectResponse


In [89]:
def get_mlflow(
            token: str = "",numofword: int = 100,
            cvmode: str ="KFold",
            nsplit: int = 5,
            n_candidate: int = 5
            ):
    
    tfidf_arr=np.load(static_path +'tfidf_arr.npy')
    label_arr=np.load(static_path +'label_arr.npy')
    label_names=np.load(static_path +'label_names.npy')
    unique_word = np.load(static_path +'unique_names.npy')

    if cvmode == "KFold":# kfoldでクロスバリデーションを行う
        CrossVaild = StratifiedKFold(n_splits=nsplit, shuffle=True)#,random_state=42)
    elif cvmode == "LeaveOneOut":# 1 vs 1 でクロスバリデーションを行う
        CrossVaild = LeaveOneOut()

    #predict-proba が使えるモデルを使う
    clf =  LogisticRegression(class_weight='balanced',max_iter=200,tol = 1e-2)

    ext = []
    mlresult =[]
    mlresult2=[]

    for train_index, test_index in tqdm(CrossVaild.split(tfidf_arr, label_arr)):
        x_train, x_test = tfidf_arr[train_index], tfidf_arr[test_index]
        y_train, y_test = label_arr[train_index], label_arr[test_index]

    #ms = StandardScaler()
        ms = MinMaxScaler()
        X_train = ms.fit_transform(x_train)
        X_test = ms.transform(x_test)

        clf,ostr = benchmark(clf,X_train, X_test, y_train, y_test)

#def print_top10(feature_names, clf, class_labels):
        if hasattr(clf, "coef_"):
            #ostr["dimensionality"] = "%d" % clf.coef_.shape[1] 
            #ostr["density"] = "%f" % density(clf.coef_) 
            ostr["top10"] = print_top10(numofword,unique_word, clf, label_names)
            intercept = clf.intercept_.tolist()
            mlresult.append(ostr)
            mlresult2.append( dict(zip(label_names,intercept))) #mlresult.append(ostr)
        
        pred_proba_train = clf.predict_proba(X_train)
        pred_proba_test = clf.predict_proba(X_test)

        df = pd.read_pickle(static_path + token)
 
        labels = set()   # 空のセットを生成(set([]))
        for i,value in enumerate(test_index):
            try:
                labels.add( df.loc[value].ai_train_label)
            except:
                pdb.set_trace()
        
        labels = list(labels) #listに戻す

        #pdb.set_trace()

        for ind, value in enumerate(test_index):
            output = {"id": str(uuid.uuid4())[:6]}
            output.update({"group": len(mlresult)-1})  #labels.index( df.loc[value].ai_train_label)})
            num = df.loc[value].koukai_number
            output.update({"querystr": num})
            output.update({"ai_train_label": df.loc[value].ai_train_label})
            output.update(
            {"ai_predict_label(pf)": df.loc[value].ai_predict_label})
            output.update(
            {"ai_predict_score(pf)": df.loc[value].ai_predict_score})
            
            ds = {key: val for key, val in zip(label_names, map(
            '{:f}'.format, pred_proba_test[ind].tolist()))}
            #pdb.set_trace()
            max_kv = max(ds.items(), key=lambda x: x[1])
            output.update({"ai_predict_label": max_kv[0]})
            output.update({"ai_predict_score": max_kv[1]})
            ds_sel = dict( sorted(ds.items(),key=lambda x:x[1],reverse=True)[0:n_candidate])
            output.update({"ai_predict_candidate": json.dumps(ds_sel)})
            
            #pdb.set_trace()
            
            ext.append(output)
    

    return {"crossvalid":ext,"mlresult":mlresult,"mlresult2":mlresult2}


In [90]:
def mlFlow2(token,num_word,cvmode,nsplit,n_candidate):
    url = 'http://172.26.106.79:7999/calc_crossvalid/'
        

   #get送信
    
    res = get_mlflow(token, num_word,
            cvmode,
            nsplit,
            n_candidate)  #requests.get( url,  params=payload)   #dataを指定する    )

    json_item_data = res
        
    #戻り値はJSON形式　dataframeと、mlresultの２つのデータ保存する
    
    
    #pdb.set_trace()
    df=json_item_data["crossvalid"]
    mlresult = json_item_data["mlresult"]
    mlresult2 = json_item_data["mlresult2"]

    cust_df = pd.json_normalize(df)
    stjson = cust_df.to_dict("records") 

    return stjson,mlresult,mlresult2

In [93]:
from typing import Tuple
import numpy as np
from sklearn import metrics
from time import time
import pdb
from typing import Tuple
from sklearn.metrics import confusion_matrix
import uuid

def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

#print_top10(unique_word, clf, label_names)
def print_top10(topN,feature_names, clf, class_labels):
    
    
    restr = {}

    if len(class_labels)>2:
        N= -1 * topN
        for i, class_label in enumerate(class_labels):
            
            top10 = np.argsort(clf.coef_[i])[N:]
            top10 = top10[::-1]
            top10_vec = np.sort(clf.coef_[i])[N:]
            top10_vec = top10_vec[::-1]
            dic = {key: val for key, val in zip(feature_names[top10],top10_vec)}
            restr[class_label] = dic
            
            #print("{} {} {}".format( class_label,len(top10),len(top10_vec)))

        return restr
    else:
        N= -1 * topN
        
        top10 = np.argsort(clf.coef_[0])[N:]
        top10 = top10[::-1]
        top10_vec = np.sort(clf.coef_[0])[N:]
        top10_vec = top10_vec[::-1]
        dic = {key: val for key, val in zip(feature_names[top10],top10_vec)}
        restr[class_labels[1]] = dic
        #print("{} {} {}".format( class_labels[0],len(top10),len(top10_vec)))

        top10 = np.argsort(clf.coef_[0])[:topN]
        top10_vec = np.sort(clf.coef_[0])[:topN]
        dic = {key: val * -1 for key, val in zip(feature_names[top10],top10_vec)}
        restr[class_labels[0]] = dic
        #print("{} {} {}".format( class_labels[1],len(top10),len(top10_vec)))

        return restr

def benchmark(clf,X_train, X_test, y_train, y_test):
    outstr = {}

    t0 = time()
    clf.fit(X_train, y_train)
    
    train_time = time() - t0
    outstr["train time"] = "%0.3f" % train_time 

    t0 = time()
    pred = clf.predict(X_test)

    #print(confusion_matrix(y_test, pred))

    test_time = time() - t0
    outstr["test time"] = "%0.3f" % test_time 
    
#正解率
    score = metrics.accuracy_score(y_test, pred)
    outstr["accuracy"] = "%0.3f" % score 

#再現性
    recall = metrics.recall_score(y_test, pred, average='macro')
    outstr["recall"] = "%0.3f" % recall

    return clf,outstr


In [94]:
crossvalid, result, result2 = mlFlow2("demo",30,"KFold",5,5)

5it [00:01,  4.60it/s]


In [95]:
result[1]

{'train time': '0.274',
 'test time': '0.002',
 'accuracy': '0.979',
 'recall': '0.970',
 'top10': {'BATERY/CYL': {'電池': 0.2218572506196054,
   '正極': 0.20630671020134872,
   '負極': 0.20553821777761908,
   '二次電池': 0.11641752364064807,
   '充電': 0.11246126714073315,
   'リチウム': 0.09861351569234594,
   '負極活物質': 0.08969010121441429,
   '正極活物質': 0.08930942652654338,
   '電力': 0.0886790833809724,
   '充放電': 0.08736680609801452,
   '蓄電': 0.0871058365334349,
   '電解液': 0.0865985715128506,
   '電源': 0.08240328628052207,
   '電池パツク': 0.08159562496214033,
   '保護': 0.07703353859521546,
   'セパレータ': 0.07648683251406366,
   '放電': 0.07205625984178239,
   '正極集電体': 0.07102333575551764,
   '吸蔵': 0.07077460958691087,
   '負極集電体': 0.06966316675900021,
   '電解質': 0.06911568946016067,
   '応用': 0.06859797698191615,
   '負極活物質層': 0.06843556184448621,
   '正極活物質層': 0.06643629994444657,
   '電子機器': 0.06529698427284208,
   'LI': 0.06524198965123444,
   '電圧': 0.06502318988844619,
   'モータ': 0.06468006054728925,
   '電動工具': 0.063

# Pytorch 

In [96]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from sklearn import metrics
import torch.nn.functional as nnf

In [97]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

#torch.set_default_tensor_type(torch.DoubleTensor)
        
static_path= ""

#device = 'cuda:0'
device = 'cpu'

1.13.0
False
0


In [98]:
class LogisticRegression(torch.nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LogisticRegression,self).__init__()
        self.l1 = torch.nn.Linear(input_dim,output_dim)
        
    def forward(self,x):
        x=self.l1(x)
        return x

In [99]:
tfidf_arr=np.load(static_path +'tfidf_arr.npy')
label_array=np.load(static_path +'label_array.npy')
label_names=np.load(static_path +'label_names.npy')
unique_word = np.load(static_path +'unique_names.npy')


In [100]:
from sklearn.model_selection import train_test_split,LeaveOneOut, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from pycm import *
import pdb

token = ""
numofword = 100
cvmode ="KFold"
nsplit = 5
n_candidate = 5
CrossVaild = StratifiedKFold(n_splits=nsplit, shuffle=True)

In [101]:
from typing import Tuple
import numpy as np
from sklearn import metrics
from time import time
import pdb
from typing import Tuple
from sklearn.metrics import confusion_matrix
import uuid
import pandas as pd
import json


def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

#print_top10(unique_word, clf, label_names)
def print_top10_torch(topN,feature_names, clf, class_labels):
    
    restr = {}

    N= -1 * topN
    
    for i, class_label in enumerate(class_labels):
        top10_vec,top10 = torch.topk(clf.l1.weight[i],topN)
            
        top10=top10.tolist()
        top10_vec=top10_vec.tolist()
            
        dic = {key: val for key, val in zip(feature_names[top10],top10_vec)}
        restr[class_label] = dic
            
    return restr

In [104]:
outstr ={}

ext = []
mlresult =[]
mlresult2=[]
token = "demo"
best = 5
n_candidate = 3

for train_index, test_index in tqdm(CrossVaild.split(tfidf_arr, label_array)):
    x_train, x_test = tfidf_arr[train_index], tfidf_arr[test_index]
    y_train, y_test = label_array[train_index], label_array[test_index]

    ms = MinMaxScaler()
    x_train = ms.fit_transform(x_train)
    x_test = ms.transform(x_test)

    X_train = torch.from_numpy(x_train.astype(np.float32)).to(device)
    X_test = torch.from_numpy(x_test.astype(np.float32)).to(device)
    Y_train = torch.from_numpy(y_train).long().to(device)
    Y_test = torch.from_numpy(y_test).long().to(device)

    train = TensorDataset(X_train,Y_train)
    train_loader = DataLoader(train,shuffle=True,batch_size=1000)
    
    test = TensorDataset(X_test,Y_test)
    test_loader = DataLoader(test,shuffle=True,batch_size=1000)
    
    input_dim = tfidf_arr.shape[1]
    output_dim = len(label_names)
    model = LogisticRegression(input_dim,output_dim)

    
    model.to(device)
    
    lr_rate = 0.5
    
    criterion = torch.nn.CrossEntropyLoss()
    
    #optimizer = torch.optim.SGD(model.parameters(),lr=lr_rate)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr_rate)
    
    loss_history = []

    iterations =0
    
    for epoch in range(20):
        total_loss = 0
        for x, y in train_loader:

        # 学習ステップ
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        
            total_loss += loss.item()
        
        loss_history.append(total_loss)
        if (epoch +1) % 100 == 0:
            print(epoch + 1, total_loss)
            
    for x_test,y_test in test_loader:
        outputs = model(x_test)
        pred_proba_vec,pred_proba= torch.topk(outputs.data,dim=1,k=n_candidate)
        _,pred= torch.max(outputs.data,dim=1)
        
        #正解率
        score = metrics.accuracy_score(y_test, pred)
        outstr["accuracy"] = "%0.3f" % score 

#再現性
        recall = metrics.recall_score(y_test, pred, average='macro')
        outstr["recall"] = "%0.3f" % recall
        
        outstr["top10"] = print_top10_torch(numofword,unique_word, model, label_names)
        intercept = model.l1.bias.tolist()
        
        mlresult.append(outstr)
        mlresult2.append( dict(zip(label_names,intercept))) #mlresult.append(ostr)
              
        df = pd.read_pickle(static_path + token)
 
        labels = set()   # 空のセットを生成(set([]))
        for i,value in enumerate(test_index):
            try:
                labels.add( df.loc[value].ai_train_label)
            except:
                pdb.set_trace()
        
        labels = list(labels) #listに戻す

        for ind, value in enumerate(test_index):
            output = {"id": str(uuid.uuid4())[:6]}
            output.update({"group": len(mlresult)-1})  #labels.index( df.loc[value].ai_train_label)})
            num = df.loc[value].koukai_number
            output.update({"querystr": num})
            output.update({"ai_train_label": df.loc[value].ai_train_label})
            output.update(
            {"ai_predict_label(pf)": df.loc[value].ai_predict_label})
            output.update(
            {"ai_predict_score(pf)": df.loc[value].ai_predict_score})

            ds = {key: val for key, val in zip(label_names[pred_proba[ind].tolist()], pred_proba_vec[ind].tolist())}
                  
            max_kv = max(ds.items(), key=lambda x: x[1])
            output.update({"ai_predict_label": max_kv[0]})
            output.update({"ai_predict_score": max_kv[1]})
            ds_sel = dict( sorted(ds.items(),key=lambda x:x[1],reverse=True)[0:n_candidate])
            output.update({"ai_predict_candidate": json.dumps(ds_sel)})

            ext.append(output)
    
    #print({"crossvalid":ext,"mlresult":mlresult,"mlresult2":mlresult2})
   

5it [00:00,  5.36it/s]


In [106]:
mlresult

[{'accuracy': '0.957',
  'recall': '0.933',
  'top10': {'BATERY/CYL': {'本実施例': 3.7118148803710938,
    'ゴム': 3.607475996017456,
    '支持台': 3.511573076248169,
    '用意': 3.5074048042297363,
    '列挙': 3.400890350341797,
    'ハウジング内': 3.3610124588012695,
    '意図': 3.28849458694458,
    'ボルト': 3.2435882091522217,
    '直線': 3.1761090755462646,
    '囲う': 3.1697580814361572,
    '外部接続端子': 3.1517083644866943,
    '延伸': 3.1192760467529297,
    '本体': 3.1175003051757812,
    '遮': 3.0496368408203125,
    '防水シート': 3.0496039390563965,
    'シリコーンゴム': 3.0493788719177246,
    '公称': 3.0488481521606445,
    'スポンジ': 3.048602342605591,
    '微細': 3.04850435256958,
    '弾性シート': 3.04734468460083,
    '従属': 3.046764850616455,
    '柔軟': 3.0466315746307373,
    'ホルダカバ': 3.0452094078063965,
    '非限定': 3.0447323322296143,
    '座': 3.0443131923675537,
    'ポリエチレンフオーム': 3.0394446849823,
    '集団': 3.0383334159851074,
    'スポンジシート': 3.037716865539551,
    'ラミネート加工': 3.036623239517212,
    '代表': 3.0362942218780518,
    